In [16]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
#tf.enable_v2_behavior()
# Import tensornetwork
import tensornetwork as tn
# Set the backend to tesorflow
# (default is numpy)
tn.set_default_backend("tensorflow")

In [17]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 0-255の値が入っているので、0-1に収まるよう正規化します
x_train, x_test = x_train / 255.0, x_test / 255.0

In [18]:
print(x_train.shape, x_test.shape)

(60000, 28, 28) (10000, 28, 28)


In [19]:
x_vec = []
for i in range(x_train.shape[0]):
    x_vec_ = np.concatenate([x_train[i][j, ::-2*(j%2)+1] for j in range(x_train.shape[1])])
    x_vec.append(x_vec_)
x_train_1d = np.vstack(x_vec)

x_vec = []
for i in range(x_test.shape[0]):
    x_vec_ = np.concatenate([x_test[i][j, ::-2*(j%2)+1] for j in range(x_test.shape[1])])
    x_vec.append(x_vec_)
x_test_1d = np.vstack(x_vec)

In [20]:
x_train_1d.shape

(60000, 784)

In [161]:
# Retrieving a component

def block(*dimensions):
    '''Construct a new matrix for the MPS with random numbers from 0 to 1'''
    size = tuple([x for x in dimensions])
    return np.random.random_sample(size) / np.max(size)**0.7

def create_MPS(rank, dim, bond_dim):
    '''Build the MPS tensor'''
    mps = [
        tn.Node( block(dim, bond_dim) )] + \
        [tn.Node( block(bond_dim, dim, bond_dim)) for _ in range(rank-2)] + \
        [tn.Node( block(bond_dim, dim) )
        ]

    #connect edges to build mps
    connected_edges=[]
    conn=mps[0][1]^mps[1][0]
    connected_edges.append(conn)
    for k in range(1,rank-1):
        conn=mps[k][2]^mps[k+1][0]
        connected_edges.append(conn)

    return mps, connected_edges

def create_MPS_labeled(rank, dim, bond_dim):
    '''Build the MPS tensor'''
    half = np.int((rank - 2) / 2)
    mps = [
        tn.Node( block(dim, bond_dim) )] + \
        [tn.Node( block(bond_dim, dim, bond_dim)) for _ in range(half)] + \
        [tn.Node( block(bond_dim, label_dim, bond_dim) )] + \
        [tn.Node( block(bond_dim, dim, bond_dim)) for _ in range(half, rank-2)] + \
        [tn.Node( block(bond_dim, dim) )
        ]

    #connect edges to build mps
    connected_edges=[]
    conn=mps[0][1]^mps[1][0]
    connected_edges.append(conn)
    for k in range(1,rank):
        conn=mps[k][2]^mps[k+1][0]
        connected_edges.append(conn)

    return mps, connected_edges


In [162]:
def feature_map(p):
    phi = [1-p, p]
    return phi

def data_tensorize(vec):
    data_tensor = [tn.Node(feature_map(p)) for p in vec]
    return data_tensor

In [163]:
label_len = 1
label_dim = 10
data_len = x_train_1d.shape[1]
rank = data_len
dim = 2
bond_dim = 10
# mps, edges = create_MPS(rank, dim, bond_dim)
mps, edges = create_MPS_labeled(rank, dim, bond_dim)

test_vec = x_train_1d[0]
data_tensor = data_tensorize(test_vec)

edges.append(data_tensor[0][0] ^ mps[0][0])
half_len = np.int(len(data_tensor) / 2)
[edges.append(data_tensor[i][0] ^ mps[i][1]) for i in range(1, half_len)]
[edges.append(data_tensor[i-label_len][0] ^ mps[i][1]) for i in range(half_len + label_len, data_len + label_len)]
for k in reversed(range(len(edges))):
    A = tn.contract(edges[k])
result = A.tensor

In [164]:
print(A.tensor.numpy())
print(A.tensor.numpy().astype("float32"))

[0.00073183 0.00070497 0.00083472 0.00063803 0.0007883  0.00069745
 0.00083175 0.00074088 0.00067797 0.00072349]
[0.00073183 0.00070497 0.00083472 0.00063803 0.0007883  0.00069745
 0.00083175 0.00074088 0.00067797 0.00072349]


In [165]:
def block(*dimensions):
    '''Construct a new matrix for the MPS with random numbers from 0 to 1'''
    size = tuple([x for x in dimensions])
    return tf.Variable(
        tf.random.uniform(shape=size, dtype=tf.dtypes.float32, maxval= 1/ np.max(size)**0.7),
        trainable=True)

def create_blocks(rank, dim, bond_dim, label_dim):
    half = np.int((rank - 2) / 2)
    blocks = [
        block(dim, bond_dim) ] + \
        [ block(bond_dim, dim, bond_dim) for _ in range(half)] + \
        [ block(bond_dim, label_dim, bond_dim) ] + \
        [ block(bond_dim, dim, bond_dim) for _ in range(half, rank-2)] + \
        [ block(bond_dim, dim) 
        ]
    return blocks

def create_MPS_labeled(blocks, rank, dim, bond_dim):
    '''Build the MPS tensor'''
    half = np.int((rank - 2) / 2)
    mps = []
    for b in blocks:
        mps.append(tn.Node(b))

    #connect edges to build mps
    connected_edges=[]
    conn=mps[0][1]^mps[1][0]
    connected_edges.append(conn)
    for k in range(1,rank):
        conn=mps[k][2]^mps[k+1][0]
        connected_edges.append(conn)

    return mps, connected_edges

class TNLayer(tf.keras.layers.Layer):
    def __init__(self, input_len, label_num, bond_dim):
        self.label_len = 1
        self.label_dim = label_num
        self.rank = input_len
        self.dim = 2
        self.bond_dim = bond_dim
        #super(TNLayer, self).__init__()
        super().__init__()
        # Create the variables for the layer.
        self.blocks = create_blocks(self.rank, self.dim, self.bond_dim, self.label_dim)

    def call(self, inputs):
        def f(input_vec, blocks, rank, dim, bond_dim, label_len):
            mps, edges = create_MPS_labeled(blocks, rank, dim, bond_dim)
            data_tensor = []
            for p in tf.unstack(input_vec):
                data_tensor.append(tn.Node([1-p, p]))
            edges.append(data_tensor[0][0] ^ mps[0][0])
            half_len = np.int(rank / 2)
            [edges.append(data_tensor[i][0] ^ mps[i][1]) for i in range(1, half_len)]
            [edges.append(data_tensor[i-label_len][0] ^ mps[i][1]) \
                 for i in range(half_len + label_len, rank + label_len)]
            for k in reversed(range(len(edges))):
                A = tn.contract(edges[k])
            result = A.tensor
            return result

        result = tf.vectorized_map(
        lambda vec: f(vec, self.blocks, self.rank, self.dim, self.bond_dim, self.label_len), inputs)
        return result

In [166]:
N = x_train_1d.shape[1]
label_len = 1
label_num = 10
data_len = x_train_1d.shape[1]
rank = data_len
dim = 2
bond_dim = 10

tn_model = tf.keras.Sequential(
    [
        tf.keras.Input(shape=(N,)),
        TNLayer(N, label_num, bond_dim),
        tf.keras.layers.Softmax()
    ])
tn_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
tn_layer_4 (TNLayer)         (None, 10)                157440    
_________________________________________________________________
softmax_4 (Softmax)          (None, 10)                0         
Total params: 157,440
Trainable params: 157,440
Non-trainable params: 0
_________________________________________________________________


In [167]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
tn_model.compile(optimizer=optimizer, 
                 loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
                 metrics=['accuracy'])
tn_model.fit(x_train_1d, y_train, epochs=300, verbose=1)

Epoch 1/300
1875/1875 [==============================] - 94s 50ms/step - loss: 2.1834 - accuracy: 0.1118
Epoch 2/300
1875/1875 [==============================] - 95s 51ms/step - loss: 2.2195 - accuracy: 0.1139
Epoch 3/300
1875/1875 [==============================] - 95s 51ms/step - loss: 2.2228 - accuracy: 0.1266
Epoch 4/300
1875/1875 [==============================] - 95s 51ms/step - loss: 2.2124 - accuracy: 0.1142
Epoch 5/300
1875/1875 [==============================] - 94s 50ms/step - loss: 2.0559 - accuracy: 0.1033
Epoch 6/300
1875/1875 [==============================] - 94s 50ms/step - loss: 2.2577 - accuracy: 0.1399
Epoch 7/300
1875/1875 [==============================] - 95s 51ms/step - loss: 2.2305 - accuracy: 0.1514
Epoch 8/300
1875/1875 [==============================] - 95s 50ms/step - loss: 2.2506 - accuracy: 0.1225
Epoch 9/300
1875/1875 [==============================] - 95s 51ms/step - loss: 2.3026 - accuracy: 0.1254
Epoch 10/300
1875/1875 [==============================]

KeyboardInterrupt: 

In [12]:
tf.config.list_physical_devices('GPU')

[]

In [168]:
# Add a channels dimension
x_train_1d = x_train_1d.astype("float32")
x_test_1d = x_test_1d.astype("float32")
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train_1d, y_train)).shuffle(60000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test_1d, y_test)).batch(32)

In [197]:
from tensorflow.keras import Model

class MyModel(Model):
    def __init__(self, input_len, label_num, bond_dim):
        super(MyModel, self).__init__()
        self.label_len = 1
        self.label_dim = label_num
        self.rank = input_len
        self.dim = 2
        self.bond_dim = bond_dim
        self.blocks = create_blocks(self.rank, self.dim, self.bond_dim, self.label_dim)
        self.softmax = tf.keras.layers.Softmax()

    def call(self, inputs):
        def f(input_vec, blocks, rank, dim, bond_dim, label_len):
            mps, edges = create_MPS_labeled(blocks, rank, dim, bond_dim)
            data_tensor = []
            for p in tf.unstack(input_vec):
                data_tensor.append(tn.Node([1-p, p]))
            edges.append(data_tensor[0][0] ^ mps[0][0])
            half_len = np.int(rank / 2)
            [edges.append(data_tensor[i][0] ^ mps[i][1]) for i in range(1, half_len)]
            [edges.append(data_tensor[i-label_len][0] ^ mps[i][1]) \
                 for i in range(half_len + label_len, rank + label_len)]
            for k in reversed(range(len(edges))):
                A = tn.contract(edges[k])
            result = A.tensor
            return self.softmax(result)

        result = tf.vectorized_map(
        lambda vec: f(vec, self.blocks, self.rank, self.dim, self.bond_dim, self.label_len), inputs)
        return result

In [198]:
N = x_train_1d.shape[1]
label_len = 1
label_num = 10
data_len = x_train_1d.shape[1]
rank = data_len
dim = 2
bond_dim = 10

model = MyModel(N, label_num, bond_dim)

In [199]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)

optimizer = tf.keras.optimizers.Adam()

In [200]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [201]:
#@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    print('gradients sample:', gradients[0].numpy())
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

In [202]:
@tf.function
def test_step(images, labels):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [203]:
EPOCHS = 5

for epoch in range(EPOCHS):
    # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1,
                        train_loss.result(),
                        train_accuracy.result() * 100,
                        test_loss.result(),
                        test_accuracy.result() * 100))

gradients sample: [[3.6746396e-06 3.3691854e-06 3.1413842e-06 2.8855334e-06 3.9668039e-06
  3.0522365e-06 3.1187069e-06 3.3026095e-06 2.9956864e-06 2.8563418e-06]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00]]
gradients sample: [[5.7549914e-06 5.2766363e-06 4.9187611e-06 4.5194179e-06 6.2101740e-06
  4.7793137e-06 4.8833635e-06 5.1697107e-06 4.6896384e-06 4.4687740e-06]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00]]
gradients sample: [[-9.2146763e-07 -8.4487743e-07 -7.8728436e-07 -7.2371444e-07
  -9.9373153e-07 -7.6499981e-07 -7.8164118e-07 -8.2705776e-07
  -7.5034848e-07 -7.1432862e-07]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
   0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
   0.0000000e+00  0.0000000e+00]]
gradients sample: [[-2.3355683e-06 -2.1414346e-06

KeyboardInterrupt: 

In [204]:
(image, label) = (x_train_1d[0], y_train[0])
with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
gradients = tape.gradient(loss, model.trainable_variables)

In [206]:
predictions

<tf.Tensor: shape=(32, 10), dtype=float32, numpy=
array([[0.10000021, 0.09999987, 0.1       , 0.10000007, 0.10000008,
        0.09999987, 0.10000004, 0.10000004, 0.09999996, 0.0999999 ],
       [0.10000028, 0.09999982, 0.09999999, 0.10000009, 0.1000001 ,
        0.09999983, 0.10000005, 0.10000005, 0.09999993, 0.09999987],
       [0.10000018, 0.09999987, 0.09999999, 0.10000005, 0.10000006,
        0.09999988, 0.10000002, 0.10000002, 0.09999996, 0.0999999 ],
       [0.10000017, 0.0999999 , 0.10000001, 0.10000006, 0.10000007,
        0.0999999 , 0.10000004, 0.10000004, 0.09999997, 0.09999993],
       [0.10000011, 0.09999991, 0.09999999, 0.10000003, 0.10000004,
        0.09999991, 0.10000001, 0.10000001, 0.09999996, 0.09999993],
       [0.10000022, 0.09999985, 0.09999999, 0.10000007, 0.10000008,
        0.09999986, 0.10000004, 0.10000004, 0.09999995, 0.0999999 ],
       [0.10000015, 0.0999999 , 0.1       , 0.10000005, 0.10000005,
        0.0999999 , 0.10000002, 0.10000002, 0.09999996, 0.09

In [184]:
loss_object(labels, predictions)

<tf.Tensor: shape=(), dtype=float32, numpy=2.3025851>